In [1]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html, datetime
import openpyxl
from openpyxl import Workbook 
from os import listdir
from os.path import isfile, join
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender 
from lib.script_generator import script_generator   
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

In [45]:
pl=sql_todf("""
SELECT UPPER(LTRIM(RTRIM(DisplayValue))) [Selections], PK_PickList [ML_Value], PK_PickListType [PLT]
FROM vwPickLists  
WHERE PK_PickListType > 410
ORDER BY PK_PickListType DESC
""", connstr)  
s=list(pl.loc[pl['PLT']==453]['Selections'])
pl['Selections'] = pl['Selections'].apply(lambda x: re.sub('[^A-Za-z0-9\s]','',str(x)))
#print(s)

pr = picklist_recommender(ctx.config['connstr'], reset_cache=True, picklist_where=' PK_PickListType > 410 ') 
#pr.normalize(' '.join(s))
pl.loc[pl['PLT']==453]

,Selections,ML_Value,PLT
37,ATTEMPTS TO AUTHENTICATE TO NONEXISTENT ACCOUNTS,4861,453
38,ATTEMPTS TO AUTHENTICATE TO INACTIVE DISABLED ...,4862,453
39,ATTEMPTS TO RESET OR CHANGE ACCOUNT CREDENTIALS,4863,453
40,ATTEMPTS TO INTERACTIVELY LOG IN TO NONPERSON ...,4864,453
41,ATTEMPTS TO INTERACTIVELY LOG IN TO SYSTEM ACC...,4865,453
42,ATTEMPTS TO AUTHENTICATE USING INVALID OR DISA...,4866,453
43,OTHER,4867,453
44,UNKNOWN,4868,453


In [29]:
path=r'C:\Users\timko\Downloads\Questionnaire\\'
files = [f for f in listdir(path) if re.search('^HVA.*1\.xlsx', f)]
print(files)
xlfile=files[5]
metric='MD9a'
#xlsx= openpyxl.load_workbook(f'{path}{xlfile}')
xlfile 
 

['HVA Asset Management Questionnaire v0.1.xlsx', 'HVA Business Continuity Management Questionnaire v0.1.xlsx', 'HVA Data Protection Questionnaire v0.1.xlsx', 'HVA Endpoint Protection Questionnaire v0.1.xlsx', 'HVA Identity and Access Management Questionnaire v0.1.xlsx', 'HVA Monitoring and Detection Questionnaire v0.1.xlsx', 'HVA Network Architecture Questionnaire v0.1.xlsx', 'HVA Remote Access Questionnaire v0.1.xlsx', 'HVA Vulnerability Management Questionnaire v0.1.xlsx']


'HVA Monitoring and Detection Questionnaire v0.1.xlsx'

In [30]:
converts={
    'Selections':lambda x: re.sub('\d{,2}\.','',str(x)).upper().strip()
    , 'ML':lambda x: str(x).strip()
    , 'SML':lambda x: re.sub('[^A-Z0-9]','',str(x)).strip()  }
df=pd.read_excel(f'{path}{xlfile}', converters=converts,  sheet_name=4, header=2, usecols='B:I').reset_index()
df = df.rename(columns={'ML Score': 'ML_Score','ML Scored': 'ML_Scored' })
for c in ['ID','Question','SML']:
    df[c] = df[c].replace(to_replace=[0, '0'], method='ffill')
if not 'ML' in df.columns:
    df['ML']=df["ML_Score"].apply(lambda x: str(x).strip())
    df['ML_Score']=df["ML_Scored"]
 
df["ID"].fillna(method='ffill', inplace=True) 
df.drop(columns=['Check Answer', 'index'], inplace=True)
df.loc[df['ID']=='MD9a']
 


,ID,Question,Selections,SML,ML,ML_Score,ML_Scored
77,MD9a,"If Yes to MD9, what types of failed authentica...",ATTEMPTS TO AUTHENTICATE TO NONEXISTENT ACCOUNTS,SML1,1,0.333333,0
78,MD9a,"If Yes to MD9, what types of failed authentica...","ATTEMPTS TO AUTHENTICATE TO INACTIVE, DISABLED...",SML1,1,0.333333,0
79,MD9a,"If Yes to MD9, what types of failed authentica...",ATTEMPTS TO INTERACTIVELY LOG IN TO NON-PERSON...,SML2,2,0.333333,0
80,MD9a,"If Yes to MD9, what types of failed authentica...",ATTEMPTS TO INTERACTIVELY LOG IN TO SYSTEM ACC...,SML2,2,0.333333,0
81,MD9a,"If Yes to MD9, what types of failed authentica...",ATTEMPTS TO RESET OR CHANGE ACCOUNT CREDENTIALS,SML1,1,0.333333,0
82,MD9a,"If Yes to MD9, what types of failed authentica...",ATTEMPTS TO AUTHENTICATE USING INVALID OR DISA...,SML2,2,0.333333,0
83,MD9a,"If Yes to MD9, what types of failed authentica...",OTHER,SML2,1,0,0
84,MD9a,"If Yes to MD9, what types of failed authentica...",UNKNOWN,SML2,1,0.001,0


In [40]:
df = df.loc[ df["ID"].str.contains('^\w\w\d(\w){0,5}$', na=False, regex=True) ]  
df = df.loc[ df["ML"].str.contains('\d', na=False,  regex=True, case=False) ]  
df['ML_Score'] = df['ML_Score'].apply(lambda s: re.sub('[^0-9\.]','',str(s)))
df['Selections'] = df['Selections'].apply(lambda x: re.sub('[^A-Za-z0-9\s]',' ',str(x)).replace('  ',' ').strip())
df.loc[df['ID']=='MD9a'] 

,ID,Question,Selections,SML,ML,ML_Score,ML_Scored
77,MD9a,"If Yes to MD9, what types of failed authentica...",ATTEMPTS TO AUTHENTICATE TO NONEXISTENT ACCOUNTS,SML1,1,0.3333333333333333,0
78,MD9a,"If Yes to MD9, what types of failed authentica...",ATTEMPTS TO AUTHENTICATE TO INACTIVE DISABLED ...,SML1,1,0.3333333333333333,0
79,MD9a,"If Yes to MD9, what types of failed authentica...",ATTEMPTS TO INTERACTIVELY LOG IN TO NON PERSON...,SML2,2,0.3333333333333333,0
80,MD9a,"If Yes to MD9, what types of failed authentica...",ATTEMPTS TO INTERACTIVELY LOG IN TO SYSTEM ACC...,SML2,2,0.3333333333333333,0
81,MD9a,"If Yes to MD9, what types of failed authentica...",ATTEMPTS TO RESET OR CHANGE ACCOUNT CREDENTIALS,SML1,1,0.3333333333333333,0
82,MD9a,"If Yes to MD9, what types of failed authentica...",ATTEMPTS TO AUTHENTICATE USING INVALID OR DISA...,SML2,2,0.3333333333333333,0
83,MD9a,"If Yes to MD9, what types of failed authentica...",OTHER,SML2,1,0,0
84,MD9a,"If Yes to MD9, what types of failed authentica...",UNKNOWN,SML2,1,0.001,0


In [43]:
dfp=df.groupby(['ID'], as_index=False).agg({'Selections':list})  
dfp['PLT']=dfp['Selections'].apply(lambda s: pr.recommend(s)['PK_PicklistType'])
pr.to_cache() 

dff=pd.merge(df,dfp.loc[:,['ID','PLT']], how='left', left_on='ID', right_on='ID')
dff=pd.merge(dff,pl, how='left', left_on=['Selections','PLT'], right_on=['Selections','PLT'])
dff.ML_Value=dff.ML_Value.fillna(0).astype('int32').astype('str')
sel = list(dff.loc[dff['ID']=='MD9a']['Selections'])
#print(' '.join(sel))
#pr.recommend(sel)
#list(pr.db_picks.loc[pr.db_picks['UsageField']=='MD9a']['DisplayValue'])

In [44]:
df1=dff
df1.SML.fillna(method='ffill', inplace=True)
df1=df1.loc[df1['ML_Score'] != '0']
df1['sortorder'] = range(1, len(df1)+1)
df1=df1.loc[:, ['ID', 'SML', 'ML','ML_Score', 'ML_Scored', 'ML_Value', 'sortorder']]
sql=SQL_INSERT_FROM_DF(df1, TARGET='@SASCORING')
with open(f'{ctx.get_dest()}script.sql', 'w') as f:
    f.write('\n'.join(sql))
df1.loc[df1['ID']=='MD9a']

--DECLARE @SASCORING AS TABLE (ID NVARCHAR(MAX), SML NVARCHAR(MAX), ML NVARCHAR(MAX), ML_Score NVARCHAR(MAX), ML_Scored NVARCHAR(MAX), ML_Value NVARCHAR(MAX), sortorder INT)


,ID,SML,ML,ML_Score,ML_Scored,ML_Value,sortorder
59,MD9a,SML1,1,0.3333333333333333,0,0,41
60,MD9a,SML1,1,0.3333333333333333,0,0,42
61,MD9a,SML2,2,0.3333333333333333,0,0,43
62,MD9a,SML2,2,0.3333333333333333,0,0,44
63,MD9a,SML1,1,0.3333333333333333,0,0,45
64,MD9a,SML2,2,0.3333333333333333,0,0,46
66,MD9a,SML2,1,0.001,0,0,47
